In [60]:
import pandas as pd

In [1]:
test_input = '''$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k'''

In [2]:
import requests

URL = "https://adventofcode.com/{year}/day/{day}/input"
url = URL.format(year='2022', day=7)
cookies = {"session":"53616c7465645f5fd345d8c12076a357c8d31b2c0c4ad1c152eefbd5c1cf09e0c8d6d7f657fc558887d443d2d07fbc90728ada9b6a91afcecb9926ffb4981a93"}
req = requests.get(url, cookies=cookies)

if req.status_code != 200:
    print("Error. Got status:", req.status_code)
    print(req.text)
    exit(3)
else:
    my_input = req.text

In [75]:
def turn_test_into_dict(this_input):
    those_files = {
        "/":{
            'isdir' : True,
            'size' : None,
            'contains' : {}
        }
    }

    cur_dir = []

    for l in this_input.split("\n"):
        if l=='':
            continue
#         print(l)
        if l[0]=="$" :
            if 'cd ..' in l :
                cur_dir = cur_dir[:-1]
            elif 'cd' in l :
                cur_dir.append(l[5:])
            else :
                pass
        else :
            sub_folder = those_files
            for d in cur_dir :
                sub_folder = sub_folder[d]['contains']

#             print("\t",cur_dir)

            if 'dir' in l :
                sub_folder[l[4:]] = {'isdir' : True, 'size' : None, 'contains' : {}}
            else :
                [size,name] = l.split(" ")
                size = int(size)
                sub_folder[name] = {
                    'isdir' : False,
                    'size' : size
                }

#             print("\t",those_files)
    return those_files

In [21]:
those_files = turn_test_into_dict(test_input)

In [76]:
those_files = turn_test_into_dict(my_input)

In [77]:
those_files['/']['contains']

{'cvt': {'isdir': True,
  'size': None,
  'contains': {'bbgsthsd': {'isdir': True,
    'size': None,
    'contains': {'djfww.fcb': {'isdir': False, 'size': 236957},
     'hcqbmwc.gts': {'isdir': False, 'size': 112286},
     'qggjrzts': {'isdir': False, 'size': 106102}}},
   'bcqrmp.czf': {'isdir': False, 'size': 146042},
   'chhdjtlw': {'isdir': True,
    'size': None,
    'contains': {'cpcfcc': {'isdir': False, 'size': 226927},
     'djfww': {'isdir': False, 'size': 309815},
     'hcqbmwc.gts': {'isdir': False, 'size': 117933},
     'mbdrgfzs': {'isdir': True,
      'size': None,
      'contains': {'hvfvt.dvq': {'isdir': False, 'size': 164331}}},
     'pbmcnpzf': {'isdir': True,
      'size': None,
      'contains': {'bjgg.cqd': {'isdir': False, 'size': 102120}}},
     'pwgswq.fld': {'isdir': False, 'size': 131558},
     'qlqqmndd.zcb': {'isdir': False, 'size': 298691}}},
   'cpcfcc': {'isdir': True,
    'size': None,
    'contains': {'ddc': {'isdir': False, 'size': 15756},
     'dqc'

In [78]:
def sum_size(this_dict):
    # this_dict is a directory dict
    if this_dict['size'] is not None :
        return this_dict['size']
    else :
        size_sum = 0
        for el in this_dict['contains'].keys():
            if this_dict['contains'][el]['size'] is not None :
                size_sum += this_dict['contains'][el]['size']
            else :
                size_sum += sum_size(this_dict['contains'][el])
        this_dict['size'] = size_sum
        return size_sum

In [79]:
for k in those_files.keys():
    sum_size(those_files[k])

In [80]:
those_files

{'/': {'isdir': True,
  'size': 46728267,
  'contains': {'cvt': {'isdir': True,
    'size': 32283214,
    'contains': {'bbgsthsd': {'isdir': True,
      'size': 455345,
      'contains': {'djfww.fcb': {'isdir': False, 'size': 236957},
       'hcqbmwc.gts': {'isdir': False, 'size': 112286},
       'qggjrzts': {'isdir': False, 'size': 106102}}},
     'bcqrmp.czf': {'isdir': False, 'size': 146042},
     'chhdjtlw': {'isdir': True,
      'size': 1351375,
      'contains': {'cpcfcc': {'isdir': False, 'size': 226927},
       'djfww': {'isdir': False, 'size': 309815},
       'hcqbmwc.gts': {'isdir': False, 'size': 117933},
       'mbdrgfzs': {'isdir': True,
        'size': 164331,
        'contains': {'hvfvt.dvq': {'isdir': False, 'size': 164331}}},
       'pbmcnpzf': {'isdir': True,
        'size': 102120,
        'contains': {'bjgg.cqd': {'isdir': False, 'size': 102120}}},
       'pwgswq.fld': {'isdir': False, 'size': 131558},
       'qlqqmndd.zcb': {'isdir': False, 'size': 298691}}},
     

In [81]:
def listing_dir (this_dict) :
    list_dir = []
    for k in this_dict.keys() :
        if this_dict[k]['isdir'] :
            list_dir.append({'dir':k,'size':this_dict[k]['size']})
        if 'contains' in this_dict[k].keys() :
            list_dir.extend(listing_dir (this_dict[k]['contains']))
    return list_dir
    

In [82]:
df_dir = pd.DataFrame(listing_dir(those_files))
df_dir

,dir,size
0,/,46728267
1,cvt,32283214
2,bbgsthsd,455345
3,chhdjtlw,1351375
4,mbdrgfzs,164331
...,...,...
164,djfww,30770
165,vhdhn,1481975
166,djfww,961278
167,zft,100941


In [83]:
sum(df_dir[df_dir['size']<=100000]['size'])

1390824

In [85]:
space_used = those_files['/']['size']
space_used

46728267

In [87]:
tot_space = 70000000
free_space = tot_space - space_used
free_space

23271733

In [89]:
need_space = 30000000
need_to_free = need_space-free_space
need_to_free

6728267

In [93]:
df_dir[df_dir['size']>=need_to_free].sort_values('size').reset_index(drop=True).loc[0,'size']

7490863